In [ ]:
#Librerias
import operator
import random
import numpy
import math
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from deap import base
from deap import benchmarks
from deap import creator
from deap import tools
from scipy.spatial.distance import euclidean
import numpy as np
from deap import algorithms
import itertools
import os
import seaborn as sns
from pandas_profiling import ProfileReport
import warnings

In [ ]:
random.seed(42)
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use("seaborn")

# Algoritmos

## GSA

In [ ]:
def GSA(minimizar=True,estancado=5, Z=benchmarks.h1,N=5, iteraciones=1000, error_parada=0.000005,size=2,pmin=-6, pmax=6, smin=-3, smax=3,Move_plot=False,verbose=True):
    
    fitness=-1 if (minimizar) else 1

    creator.create("FitnessMax", base.Fitness, weights=(1.0*fitness,))
    creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, acceleration=list,
    smin=None, smax=None, best=None,worst=None)

    toolbox = base.Toolbox()
    
    #Particle generator
    def generate(size, pmin, pmax, smin, smax):
        part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
        part.speed = [random.uniform(smin, smax) for _ in range(size)]
        part.smin = smin
        part.smax = smax
        return part
    toolbox.register("Particle", generate, size=size, pmin=pmin, pmax=pmax, smin=smin, smax=smax)
    
    #Population
    toolbox.register("population", tools.initRepeat, list, toolbox.Particle)
    
    #Particle update
    def updateParticle(part,pop,i):        
        F=[]
        for temp in range(len(pop)):
            if temp!=i:
                temp=pop[temp]
                k=(G*temp.weight*part.weight)/euclidean(temp,part)
                f_ij= [x * k for x in list(map(operator.sub, temp, part))]
                F.append(f_ij)

        w=[random.uniform(0, 1) for _ in range(len(pop)-1)]

        suma = np.zeros(len(part))
        for j in range(len(pop)-1):
            suma +=[x * w[j] for x in F[j]]
        F=suma 

        #Aceleracion
        if part.weight!=0:
            a=F/part.weight
        else:
            for temp in range(len(pop)):
                if temp!=i:
                    temp=pop[temp]
                    k=temp.weight/(euclidean(temp,part)+error)
                    a=[t * k for t in list(map(operator.sub, temp, part))]
            a*=G       
        part.speed=[random.uniform(0, 1)*part.speed[s]+a[s] for s in range(len(part.speed))]
        part[:] = list(map(operator.add, part, part.speed))    
    toolbox.register("update", updateParticle)
    
    toolbox.register("evaluate",Z)
    
    pop = toolbox.population(n=N)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals","Best_of_all","best"] + stats.fields

    best = None
    worst=None
    
    global G,error,M
    G=1
    error=0.01
    best_ant=0
    
    images = []
    
    best_particles=[]
    best_values=[]
    best_of_all=[]
    estancamiento=0
    
    for g in range(iteraciones):        
        M=0        
        evaluations=[]
        if g==0:
            best_ant=np.inf*fitness
            best_value=best_ant
        else:
            best_ant = best_value
        
        for part in pop:
            part.fitness.values = toolbox.evaluate(part)
            evaluations.append(part.fitness.values[0])
        
        if minimizar:
            best=min(evaluations)
            bestP=pop[np.argmin(np.array(evaluations))]
            best_particles.append(bestP)
            best_values.append(best)
            worst=max(evaluations)
        else:
            best=max(evaluations)
            bestP=pop[np.argmax(np.array(evaluations))]
            best_particles.append(bestP)
            best_values.append(best)
            worst=min(evaluations)
        
        if minimizar:
            best_value=min(best_values)
            best_particle=best_particles[np.argmin(np.array(best_values))]                
        else:
            best_value=max(best_values)
            best_particle=best_particles[np.argmax(np.array(best_values))]
        best_of_all.append(best_value)
        
        logbook.record(gen=g+1, evals=len(pop),Best_of_all=best_value,best=best, **stats.compile(pop))
        if verbose:
            print(logbook.stream)  
        
        if g!=0 and abs(best_value-best_ant)<error_parada:
            estancamiento+=1
            if estancamiento>=estancado:
                break
        
        if Move_plot and size==2:
            points = np.array(pop)
            fig, ax = plt.subplots()
            ax.set_title(f'Posición de partículas en el tiempo {g}')
                        
            x = points[:,0]
            y = points[:,1]
            if g!=0:
                h=np.zeros(len(x))
                h[pop.index(best_particle)]=1
                ax.scatter(x, y, c=['r' if i == 0 else 'g' for i in h])
            else:
                h=np.zeros(len(x))
                ax.scatter(x, y, c=['r' if i == 0 else 'g' for i in h])
                
            ax.set_xlim(pmin-8, pmax+8)
            ax.set_ylim(pmin-8, pmax+8)
            
            # Se guarda la figura en una imagen
            fig.canvas.draw()
            img = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
            images.append(img)
            plt.close()
            
        for part in pop:
            part.weight=(part.fitness.values[0]-worst)/(best-worst)
            M+=part.weight
            
        for part in pop:
            part.weight=part.weight/M
        
        for i in range(len(pop)):
            toolbox.update(pop[i],pop,i)
    
    if Move_plot and size==2:
        images[0].save('Particle_motion.gif', save_all=True, append_images=images[1:], duration=600, loop=0)
        
    return pop, logbook, best_particle, best_value,best_of_all

In [ ]:
pop, logbook, best_particle, best_value,best_of_all=GSA(Z=benchmarks.sphere,Move_plot=True)


## GSA MEJORA

In [ ]:
def GSA_smart_restarts(reinicios_inteligentes=5, estancado=50,minimizar=True, Z=benchmarks.h1,N=5, iteraciones=1000, error_parada=0.000005,size=2,pmin=-6, pmax=6, smin=-3, smax=3,verbose=True):
    
    fitness=-1 if (minimizar) else 1

    creator.create("FitnessMax", base.Fitness, weights=(1.0*fitness,))
    creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, acceleration=list,
    smin=None, smax=None, best=None,worst=None)

    toolbox = base.Toolbox()
    
    #Particle generator
    def generate(size, pmin, pmax, smin, smax):
        part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
        part.speed = [random.uniform(smin, smax) for _ in range(size)]
        part.smin = smin
        part.smax = smax
        return part
    toolbox.register("Particle", generate, size=size, pmin=pmin, pmax=pmax, smin=smin, smax=smax)
    
    #Population
    toolbox.register("population", tools.initRepeat, list, toolbox.Particle)
    
    #Particle update
    def updateParticle(part,pop,i):        
        F=[]
        for temp in range(len(pop)):
            if temp!=i:
                temp=pop[temp]
                k=(G*temp.weight*part.weight)/euclidean(temp,part)
                f_ij= [x * k for x in list(map(operator.sub, temp, part))]
                F.append(f_ij)

        w=[random.uniform(0, 1) for _ in range(len(pop)-1)]

        suma = np.zeros(len(part))
        for j in range(len(pop)-1):
            suma +=[x * w[j] for x in F[j]]
        F=suma 

        #Aceleracion
        if part.weight!=0:
            a=F/part.weight
        else:
            for temp in range(len(pop)):
                if temp!=i:
                    temp=pop[temp]
                    k=temp.weight/(euclidean(temp,part)+error)
                    a=[t * k for t in list(map(operator.sub, temp, part))]
            a*=G       
        part.speed=[random.uniform(0, 1)*part.speed[s]+a[s] for s in range(len(part.speed))]
        part[:] = list(map(operator.add, part, part.speed))    
    toolbox.register("update", updateParticle)
    
    #Evaluation function
    toolbox.register("evaluate",Z)
    
    #Main
    pop = toolbox.population(n=N)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["restart","gen", "bestR","Best_of_all","best"] + stats.fields

    best = None
    worst=None
    
    global G,error,M
    G=1
    error=0.01
    best_ant=0
    
    best_particles=[]
    best_values=[]
    best_of_all=[]
    generacion=[]
    best_r=0
    estancamiento=0
    
    for r in range(reinicios_inteligentes):
        for g in range(iteraciones):        
            M=0        
            evaluations=[]
            if g==0:
                best_ant=np.inf*fitness
                best_value=best_ant
            else:
                best_ant = best_value

            for part in pop:
                part.fitness.values = toolbox.evaluate(part)
                evaluations.append(part.fitness.values[0])

            if minimizar:
                best=min(evaluations)
                bestP=pop[np.argmin(np.array(evaluations))]
                best_particles.append(bestP)
                best_values.append(best)
                worst=max(evaluations)
            else:
                best=max(evaluations)
                bestP=pop[np.argmax(np.array(evaluations))]
                best_particles.append(bestP)
                best_values.append(best)
                worst=min(evaluations)
                
            generacion.append(r)
            if minimizar:
                best_value=min(best_values)
                best_particle=best_particles[np.argmin(np.array(best_values))]
                best_r=generacion[np.argmin(np.array(best_values))]
            else:
                best_value=max(best_values)
                best_particle=best_particles[np.argmax(np.array(best_values))]
                best_r=generacion[np.argmax(np.array(best_values))]
                
            best_of_all.append(best_value)
            
            logbook.record(restart=r+1,gen=g+1,bestR=best_r+1,Best_of_all=best_value,best=best,**stats.compile(pop))
            if verbose:
                print(logbook.stream)   
                
            if g!=0 and abs(best_value-best_ant)<error_parada:
                estancamiento+=1
                if estancamiento>=estancado:                    
                    break  

            for part in pop:
                part.weight=(part.fitness.values[0]-worst)/(best-worst)
                M+=part.weight

            for part in pop:
                part.weight=part.weight/M

            for i in range(len(pop)):
                toolbox.update(pop[i],pop,i)               
                             
        pop = toolbox.population(n=N)
        estancamiento=0
    
    return pop, logbook, best_particle, best_value,best_of_all

In [ ]:
pop, logbook, best_particle, best_value,best_of_all=GSA_smart_restarts(Z=benchmarks.cigar)
best_value

## PSO

In [ ]:
def PSO(minimizar=True,estancado=5, Z=benchmarks.h1,N=5, iteraciones=1000, error_parada=0.000005,phi1=2.0, phi2=2.0,size=2,pmin=-6, pmax=6, smin=-3, smax=3,verbose=True):
    
    fitness=-1 if (minimizar) else 1

    creator.create("FitnessMax", base.Fitness, weights=(1.0*fitness,))
    creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, acceleration=list,
    smin=None, smax=None, best=None,worst=None)

    toolbox = base.Toolbox()
    
    #Particle generator
    def generate(size, pmin, pmax, smin, smax):
        part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
        part.speed = [random.uniform(smin, smax) for _ in range(size)]
        part.smin = smin
        part.smax = smax
        return part
    toolbox.register("Particle", generate, size=size, pmin=pmin, pmax=pmax, smin=smin, smax=smax)
    
    #Population
    toolbox.register("population", tools.initRepeat, list, toolbox.Particle)
    
    #Particle update
    def updateParticle(part, best, phi1, phi2):
        u1 = (random.uniform(0, phi1) for _ in range(len(part)))
        u2 = (random.uniform(0, phi2) for _ in range(len(part)))
        v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
        v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
        part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
        for i, speed in enumerate(part.speed):
            if abs(speed) < part.smin:
                part.speed[i] = math.copysign(part.smin, speed)
            elif abs(speed) > part.smax:
                part.speed[i] = math.copysign(part.smax, speed)
        part[:] = list(map(operator.add, part, part.speed))
    toolbox.register("update", updateParticle, phi1=phi1, phi2=phi2)
    
    #Evaluation function
    toolbox.register("evaluate",Z)
    
    #Main
    pop = toolbox.population(n=N)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals","Best_of_all"] + stats.fields

    
    GEN = iteraciones
    best = None
    best_ant = None
    best_fitness_values = (None,)
    best_of_all = []
    estancamiento=0
    
    for g in range(GEN):
        best_ant = best_fitness_values[0]
        for part in pop:
            part.fitness.values = toolbox.evaluate(part)
            if minimizar:
                if not part.best or part.best.fitness.values[0] > part.fitness.values[0]:
                    part.best = creator.Particle(part)
                    part.best.fitness.values = part.fitness.values
                if not best or best.fitness.values[0] > part.fitness.values[0]:
                    best = creator.Particle(part)
                    best.fitness.values = part.fitness.values
                    best_fitness_values = best.fitness.values
            else:
                if not part.best or part.best.fitness.values[0] < part.fitness.values[0]:
                    part.best = creator.Particle(part)
                    part.best.fitness.values = part.fitness.values
                if not best or best.fitness.values[0] < part.fitness.values[0]:
                    best = creator.Particle(part)
                    best.fitness.values = part.fitness.values
                    best_fitness_values = best.fitness.values
        
        best_of_all.append(best_fitness_values[0])
        
        logbook.record(gen=g+1, evals=len(pop),Best_of_all=best_fitness_values[0], **stats.compile(pop))
        if verbose:
            print(logbook.stream)
        
        if g!=0 and abs(best_fitness_values[0]-best_ant)<error_parada:
            estancamiento+=1
            if estancamiento>=estancado:
                break
        
        for part in pop:
            toolbox.update(part, best)     
        
    return pop, logbook, best,best_fitness_values[0], best_of_all

In [ ]:
pop, logbook, best_particle, best_value,best_of_all=PSO(Z=benchmarks.cigar)

## PSO MEJORA

In [ ]:
def PSO_smart_restarts(reinicios_inteligentes=5, estancado=50,minimizar=True, Z=benchmarks.h1,N=5, iteraciones=1000, error_parada=0.000005,phi1=2.0, phi2=2.0,size=2,pmin=-6, pmax=6, smin=-3, smax=3,verbose=True):
    
    
    fitness=-1 if (minimizar) else 1

    creator.create("FitnessMax", base.Fitness, weights=(1.0*fitness,))
    creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, acceleration=list,
    smin=None, smax=None, best=None,worst=None)

    toolbox = base.Toolbox()
    
    #Particle generator
    def generate(size, pmin, pmax, smin, smax):
        part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
        part.speed = [random.uniform(smin, smax) for _ in range(size)]
        part.smin = smin
        part.smax = smax
        return part
    toolbox.register("Particle", generate, size=size, pmin=pmin, pmax=pmax, smin=smin, smax=smax)
    
    #Population
    toolbox.register("population", tools.initRepeat, list, toolbox.Particle)
    
    #Particle update
    def updateParticle(part, best, phi1, phi2):
        u1 = (random.uniform(0, phi1) for _ in range(len(part)))
        u2 = (random.uniform(0, phi2) for _ in range(len(part)))
        v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
        v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
        part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
        for i, speed in enumerate(part.speed):
            if abs(speed) < part.smin:
                part.speed[i] = math.copysign(part.smin, speed)
            elif abs(speed) > part.smax:
                part.speed[i] = math.copysign(part.smax, speed)
        part[:] = list(map(operator.add, part, part.speed))
    toolbox.register("update", updateParticle, phi1=phi1, phi2=phi2)
    
    #Evaluation function
    toolbox.register("evaluate",Z)
    
    #Main
    pop = toolbox.population(n=N)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["restart","gen", "bestR","Best_of_all"] + stats.fields

    estancamiento = 0
    best = None
    best_fitness_values = (None,)
    best_of_all = []
    best_r=0
    
    for r in range(reinicios_inteligentes):
        for g in range(iteraciones):
            best_ant = best_fitness_values[0]
            for part in pop:
                part.fitness.values = toolbox.evaluate(part)
                if minimizar:
                    if not part.best or part.best.fitness.values[0] > part.fitness.values[0]:
                        part.best = creator.Particle(part)
                        part.best.fitness.values = part.fitness.values
                    if not best or best.fitness.values[0] > part.fitness.values[0]:
                        best = creator.Particle(part)
                        best.fitness.values = part.fitness.values
                        best_fitness_values = best.fitness.values
                        best_r=r
                else:
                    if not part.best or part.best.fitness.values[0] < part.fitness.values[0]:
                        part.best = creator.Particle(part)
                        part.best.fitness.values = part.fitness.values
                    if not best or best.fitness.values[0] < part.fitness.values[0]:
                        best = creator.Particle(part)
                        best.fitness.values = part.fitness.values
                        best_fitness_values = best.fitness.values
                        best_r=r

            best_of_all.append(best_fitness_values[0])
            
            logbook.record(restart=r+1,gen=g+1, bestR=best_r+1,Best_of_all=best_fitness_values[0], **stats.compile(pop))
            if verbose:
                print(logbook.stream)
            
            if g!=0 and abs(best_fitness_values[0]-best_ant)<error_parada:
                estancamiento+=1
                if estancamiento>=estancado:   
                    break  

            for part in pop:
                toolbox.update(part, best)              
                        
        pop = toolbox.population(n=N)
        estancamiento=0

    return pop, logbook, best,best_fitness_values[0], best_of_all

In [ ]:
pop, logbook, best_particle, best_value,best_of_all=PSO_smart_restarts(Z=benchmarks.cigar,verbose=False)
best_value

## ALGORITMO GENÉTICO

### 1. eaSimple

In [ ]:
def eaSimple(minimizar=True, Z=benchmarks.h1,N=5, iteraciones=1000,size=2,pmin=-6, pmax=6, smin=-3, smax=3,verbose=True):
    
    fitness=-1 if (minimizar) else 1

    creator.create("FitnessMax", base.Fitness, weights=(1.0*fitness,))
    creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, acceleration=list,
    smin=None, smax=None, best=None,worst=None)

    toolbox = base.Toolbox()
    
    #Particle generator
    def generate(size, pmin, pmax, smin, smax):
        part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
        part.speed = [random.uniform(smin, smax) for _ in range(size)]
        part.smin = smin
        part.smax = smax
        return part
    toolbox.register("Particle", generate, size=size, pmin=pmin, pmax=pmax, smin=smin, smax=smax)
    
    #Population
    toolbox.register("population", tools.initRepeat, list, toolbox.Particle)
    
    toolbox.register("evaluate", Z)

    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop = toolbox.population(n=N)
    hof = tools.HallOfFame(1)
    
    pop, log= algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=iteraciones, stats=stats, halloffame=hof, verbose=verbose)
    best_particle=hof[0]
    best_value=Z(best_particle)[0]
    return pop, log, best_particle,best_value

In [ ]:
pop, log, best_particle,best_value=eaSimple(Z=benchmarks.cigar)
best_value

### 2. eaMuPlusLambda

In [ ]:
def eaMuPlusLambda(minimizar=True, Z=benchmarks.h1,N=5, iteraciones=1000,size=2,pmin=-6, pmax=6, smin=-3, smax=3,verbose=True):
    
    fitness=-1 if (minimizar) else 1

    creator.create("FitnessMax", base.Fitness, weights=(1.0*fitness,))
    creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, acceleration=list,
    smin=None, smax=None, best=None,worst=None)

    toolbox = base.Toolbox()
    
    #Particle generator
    def generate(size, pmin, pmax, smin, smax):
        part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
        part.speed = [random.uniform(smin, smax) for _ in range(size)]
        part.smin = smin
        part.smax = smax
        return part
    toolbox.register("Particle", generate, size=size, pmin=pmin, pmax=pmax, smin=smin, smax=smax)
    
    #Population
    toolbox.register("population", tools.initRepeat, list, toolbox.Particle)
    
    toolbox.register("evaluate", Z)

    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop = toolbox.population(n=N)
    hof = tools.HallOfFame(1)
    
    pop, log= algorithms.eaMuPlusLambda(pop, toolbox, cxpb=0.5, mutpb=0.2, mu=40, lambda_=60,ngen=iteraciones, stats=stats, halloffame=hof, verbose=verbose)
    best_particle=hof[0]
    best_value=Z(best_particle)[0]
    return pop, log, best_particle,best_value

In [ ]:
pop, log, best_particle,best_value=eaMuPlusLambda(Z=benchmarks.cigar)
best_value

### 3. eaMuCommaLambda

In [ ]:
def eaMuCommaLambda(minimizar=True, Z=benchmarks.h1,N=5, iteraciones=1000,size=2,pmin=-6, pmax=6, smin=-3, smax=3,verbose=True):
    
    fitness=-1 if (minimizar) else 1

    creator.create("FitnessMax", base.Fitness, weights=(1.0*fitness,))
    creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, acceleration=list,
    smin=None, smax=None, best=None,worst=None)

    toolbox = base.Toolbox()
    
    #Particle generator
    def generate(size, pmin, pmax, smin, smax):
        part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
        part.speed = [random.uniform(smin, smax) for _ in range(size)]
        part.smin = smin
        part.smax = smax
        return part
    toolbox.register("Particle", generate, size=size, pmin=pmin, pmax=pmax, smin=smin, smax=smax)
    
    #Population
    toolbox.register("population", tools.initRepeat, list, toolbox.Particle)
    
    toolbox.register("evaluate", Z)

    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop = toolbox.population(n=N)
    hof = tools.HallOfFame(1)
    
    pop, log =  algorithms.eaMuCommaLambda(pop, toolbox, cxpb=0.5, mu=40, lambda_=60,mutpb=0.2, ngen=iteraciones, stats=stats, halloffame=hof, verbose=verbose)
    best_particle=hof[0]
    best_value=Z(best_particle)[0]
    return pop, log, best_particle,best_value

In [ ]:
pop, log, best_particle,best_value=eaMuCommaLambda(Z=benchmarks.cigar)
best_value

## Análisis de surrogados

**Análisis de surrogados de GSA con reinicio inteligente para el objetivo cigar**

In [ ]:
ytrain=[]
xtrain=[]
N=[2,15,50]
size=[2,3,6]
iteraciones=[250,500,1000]
pmin_v=funciones["cigar"][2][0]
pmax_v=funciones["cigar"][2][1]
minimizar_value=funciones["cigar"][0]
z_value=funciones["cigar"][1]
combinations = list(itertools.product(N, size, iteraciones))
seed_values = [random.randint(0, 1000000) for _ in range(10)]


for N_value, size_value, iteraciones_value in combinations:
    for t in range(10):
        random.seed(seed_values[t])
        pop_v, logbook_v, best_particle_v, best_value_v, best_of_all_v = GSA_smart_restarts(minimizar=minimizar_value,Z=z_value, N=N_value, size=size_value,pmin=pmin_v,pmax=pmax_v, iteraciones=iteraciones_value,verbose=False)
        ytrain.append(best_value_v)
        xtrain.append([N_value, size_value, iteraciones_value])

In [ ]:
y=np.array(ytrain)
x=np.array(xtrain
min_id=np.argmin(y)
print(y[min_id])
print(x[min_id])

In [ ]:
from sklearn.tree import DecisionTreeRegressor 
from sklearn.svm import SVR
import xgboost as xgb
nombre_variables=["N","Size","Iteraciones"]
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, seed=24)

model.fit(x,y)
model.get_booster().feature_names = nombre_variables

In [ ]:
N_ = np.linspace(5, 100,retstep=1, dtype=int)[0].tolist()
Size_ = np.linspace(2,20,retstep=1, dtype=int)[0].tolist()
iteraciones_ = np.linspace(100, 1500,retstep=100, dtype=int)[0].tolist()

combinations = list(itertools.product(N_, Size_, iteraciones_))
matrix = np.array(combinations)

In [ ]:
ypred = model.predict(matrix)
min_idx = np.argmin(ypred)
min_x = matrix[min_idx]
min_y = ypred[min_idx]
print(min_x,min_y)

In [ ]:
pop_v, logbook_v, best_particle_v, best_value_v, best_of_all_v = GSA_smart_restarts(minimizar=minimizar_value,Z=z_value, N=34, size=2,pmin=pmin_v,pmax=pmax_v, iteraciones=100,verbose=False)
best_value_v

In [ ]:
import graphviz
xgb.plot_importance(model)
plt.show()

os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz/bin/'


graph = xgb.to_graphviz(model)
graph.render('arbol-decision')

In [ ]:
v=[]
for i in range(50):
    pop_v, logbook_v, best_particle_v, best_value_v, best_of_all_v = GSA_smart_restarts(minimizar=minimizar_value,Z=z_value, N=34, size=2,pmin=pmin_v,pmax=pmax_v, iteraciones=100,verbose=False)
    v.append(best_value_v)
std=np.array(v).std()
mean=np.array(v).mean()